In [ ]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
from glob import glob
import numpy as np
import torch.functional as F
from efficientnet_pytorch_3d import EfficientNet3D
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as T
from tqdm import tqdm_notebook
device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


데이터셋

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_glob, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_list = glob(img_glob)
        self.transform = transform
        self.target_transform = target_transform
        self.file_name_list=[]
        self.file_remove_list=[]
        self.label_list=[]
        
        for i in range(len(self.img_list)):
            self.file_name_list.append(os.path.splitext(os.path.basename(self.img_list[i]))[0][:9])

        for i in range(len(self.file_name_list)):
            label_index=self.img_labels.loc[self.img_labels['PathologyNumber'] == self.file_name_list[i]]
            if len(label_index)==0:
                self.file_remove_list.append(self.img_list[i])
            else:
                if label_index['MMR status'].to_list()[0]=='Normal':
                    self.label_list.append(0)
                else:
                    self.label_list.append(1)
        for i in range(len(self.file_remove_list)):
            self.img_list.remove(self.file_remove_list[i])
        self.image_x5=[f.replace('/2.5x_standard', '/5x_standard') for f in self.img_list]
        self.image_x10=[f.replace('/2.5x_standard', '/10x_standard') for f in self.img_list]
        img_5x_temp=[]
        img_10x_temp=[]
        for i in range(len(self.img_list)):
            for j in range(4):
                img_5x_temp.append(self.image_x5[i][:-4]+'_'+str(j)+'.jpg')
            for j in range(16):
                img_10x_temp.append(self.image_x10[i][:-4]+'_'+str(j)+'.jpg')
            self.image_x5[i]=img_5x_temp
            self.image_x10[i]=img_10x_temp
        self.transform = T.Resize(224)
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image = (1-self.transform(read_image(self.img_list[idx])))/255
        label = self.label_list[idx]
        image_5x=torch.zeros(4,3,224,224)
        image_10x=torch.zeros(16,3,224,224)
        for i in range(4):
            image_5x[i]=self.transform(read_image(self.image_x5[idx][i]))
        for i in range(16):
            image_10x[i]=self.transform(read_image(self.image_x10[idx][i]))
        image_5x=np.transpose(1-image_5x, axes=(1, 0, 2,3))/255
        image_10x=np.transpose(1-image_10x, axes=(1, 0, 2,3))/255
        image_5x=torch.reshape(1-image_5x,(3*4,224,224))
        image_10x=torch.reshape(1-image_10x,(16*3,224,224))
        
        return image,image_5x,image_10x,label

In [ ]:
img_glob='../../data/CycleGANData/3divisionTile/2.5x_standard/*.jpg'
annotations_file='../../data/OriginalData/MMR.csv'
dataset=CustomImageDataset(annotations_file,img_glob)
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, drop_last=True)

Panoptes

In [ ]:
class Panoptes(nn.Module):
    def __init__(self):
        super().__init__()
        self.model_25x=EfficientNet.from_name('efficientnet-b2', in_channels=3).to(device0)
        self.model_5x=EfficientNet.from_name('efficientnet-b2', in_channels=3*4).to(device0)
        self.model_10x=EfficientNet.from_name('efficientnet-b2', in_channels=3*16).to(device0)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(3000, 300).to(device0)
        self.fc2 = nn.Linear(300, 30).to(device0)
        self.fc3 = nn.Linear(30, 1).to(device0)
        self.cat1=torch.cat
        self.cat2=torch.cat
        self.sig = nn.Sigmoid()
    def forward(self, x25,x5,x10):
        x25 = self.model_25x(x25).to(device0)
        x5 = self.model_5x(x5).to(device0)
        x10 = self.model_10x(x10).to(device0)
        x = self.cat1((x25, x5), dim=1).to(device0)
        x = self.cat2((x, x10), dim=1).to(device0)
        x = self.relu(self.fc1(x).to(device0))
        x = self.relu(self.fc2(x).to(device0))
        x = self.fc3(x).to(device0)
        return x
    
model = Panoptes().to(device0)
criterion = nn.BCEWithLogitsLoss().to(device0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

Train

In [ ]:
MIN_loss=0
#train
for epoch in range(10000):
    cost = 0.0
    train=tqdm_notebook(train_dataloader)
    count=0
    
    TP_count=1
    FP_count=1
    TN_count=1
    FN_count=1
    for x25,x5,x10, y in train:
        y = y.to(device0)
        count+=1
        
        output = model(x25.to(device0),x5.to(device0),x10.to(device0))
        z=torch.unsqueeze(torch.squeeze(y),0).type('torch.cuda.FloatTensor')
        k=torch.unsqueeze(torch.squeeze(output),0)
        
        loss = criterion(k, z).to(device0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        for j in range(64):
            if z[0,j]==0:
                if k[0,j]>=0.5:
                    FP_count+=1
                else:
                    TN_count+=1
            else:
                if k[0,j]>=0.5:
                    TP_count+=1
                else:
                    FN_count+=1
        
        
        cost += loss
    
        train.set_description(f"epoch: {epoch+1}/{10000} Step: {count+1} loss : {cost/count:.4f} negative_accuracy: {TN_count/(FP_count+TN_count):.4f} positive_accuracy: {TP_count/(TP_count+FN_count):.4f}")
#validation
    count=0
    val=tqdm_notebook(validation_dataloader)
    TP_count=1
    FP_count=1
    TN_count=1
    val_cost=0
    FN_count=1
    model.eval()
    with torch.no_grad():
        for x25,x5,x10, y in val:
            y = y.to(device0)
            count+=1
            
            output = model(x25.to(device0),x5.to(device0),x10.to(device0))
            z=torch.unsqueeze(torch.squeeze(y),0).type('torch.cuda.FloatTensor')
            k=torch.unsqueeze(torch.squeeze(output),0)
            
            loss = criterion(k, z).to(device0)
            for j in range(16):
                if z[0,j]==0:
                    if k[0,j]>=0.5:
                        FP_count+=1
                    else:
                        TN_count+=1
                else:
                    if k[0,j]>=0.5:
                        TP_count+=1
                    else:
                        FN_count+=1
            val_cost += loss
            val.set_description(f"Validation epoch: {epoch+1}/{10000} Step: {count+1} loss : {val_cost/count:.4f} negative_accuracy: {TN_count/(FP_count+TN_count):.4f} positive_accuracy: {TP_count/(TP_count+FN_count):.4f}")

    if MIN_loss<(TP_count/(TP_count+FN_count)):
        torch.save(model.state_dict(), '../../model/3Division_class/call_back.pt')
        MIN_loss=TP_count/(TP_count+FN_count)
        
